## Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier

from jenga.basis import Dataset
from jenga.models.simple_model import SimpleModel
from jenga.corruptions.perturbations import Perturbation
from jenga.cleaning.imputation import MeanModeImputation, DatawigImputation
from jenga.cleaning.outlier_detection import PyODKNN, PyODIsolationForest

In [2]:
seed = 10

In [3]:
dataset = Dataset(seed, "credit-g")

Dataset 'credit-g', target: 'class'
**Author**: Dr. Hans Hofmann  

**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    

**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)



**German Credit data**  

This dataset classifies people described by a set of attributes as good or bad credit risks.



This dataset comes with a cost matrix: 

``` 

      Good  Bad (predicted)  

Good   0    1   (actual)  

Bad    5    0  

```



It is worse to class a customer as go

Attribute types: 


,attribute_names,categorical_indicator
0,checking_status,True
1,duration,False
2,credit_history,True
3,purpose,True
4,credit_amount,False
5,savings_status,True
6,employment,True
7,installment_commitment,False
8,personal_status,True
9,other_parties,True


In [4]:
all_data = dataset.all_data
all_data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,female div/dep/mar,none,...,real estate,31.0,none,own,1.0,unskilled resident,1.0,none,yes,good
996,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,male div/sep,none,...,life insurance,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,good
997,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,male single,none,...,car,38.0,none,own,1.0,skilled,1.0,none,yes,good
998,<0,45.0,existing paid,radio/tv,1845.0,<100,1<=X<4,4.0,male single,none,...,no known property,23.0,none,for free,1.0,skilled,1.0,yes,yes,bad


In [5]:
attribute_names = dataset.attribute_names
attribute_names

['checking_status',
 'duration',
 'credit_history',
 'purpose',
 'credit_amount',
 'savings_status',
 'employment',
 'installment_commitment',
 'personal_status',
 'other_parties',
 'residence_since',
 'property_magnitude',
 'age',
 'other_payment_plans',
 'housing',
 'existing_credits',
 'job',
 'num_dependents',
 'own_telephone',
 'foreign_worker']

In [6]:
attribute_types = dataset.attribute_types
attribute_types

,attribute_names,categorical_indicator
0,checking_status,True
1,duration,False
2,credit_history,True
3,purpose,True
4,credit_amount,False
5,savings_status,True
6,employment,True
7,installment_commitment,False
8,personal_status,True
9,other_parties,True


### Visualize the dataset

In [ ]:
## plot the original dataset
def hide_current_axis(*args, **kwds):
        plt.gca().set_visible(False)
        
def plot_data(data):
    sns.set_style("white") # grid/no grid style: darkgrid, whitegrid, dark, white, ticks
    
    plot = sns.pairplot(data, hue="class")
    plot.map_upper(hide_current_axis)
    plt.show()

In [ ]:
plot_data(all_data)

### Get training and test sets

In [7]:
train_data, train_labels, test_data, test_labels = dataset.get_train_test_data()

display(train_data.head())
print(train_labels[0:5])

display(test_data.head())
print(test_labels[0:5])

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
188,<0,12.0,existing paid,radio/tv,674.0,100<=X<500,4<=X<7,4.0,male mar/wid,none,1.0,life insurance,20.0,none,own,1.0,skilled,1.0,none,yes
194,0<=X<200,45.0,existing paid,radio/tv,3031.0,100<=X<500,1<=X<4,4.0,male single,guarantor,4.0,life insurance,21.0,none,rent,1.0,skilled,1.0,none,yes
225,no checking,36.0,no credits/all paid,repairs,2613.0,<100,1<=X<4,4.0,male single,none,2.0,car,27.0,none,own,2.0,skilled,1.0,none,yes
580,0<=X<200,18.0,critical/other existing credit,new car,1056.0,<100,>=7,3.0,male single,guarantor,3.0,real estate,30.0,bank,own,2.0,skilled,1.0,none,yes
428,no checking,9.0,existing paid,furniture/equipment,1313.0,<100,>=7,1.0,male single,none,4.0,car,20.0,none,own,1.0,skilled,1.0,none,yes


['bad' 'bad' 'good' 'bad' 'good']


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.0,delayed previously,used car,2993.0,<100,1<=X<4,3.0,male single,none,2.0,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.0,critical/other existing credit,radio/tv,3656.0,no known savings,>=7,4.0,male single,none,4.0,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,12.0,critical/other existing credit,new car,1255.0,<100,>=7,4.0,male single,none,4.0,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,8.0,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,2.0,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.0,critical/other existing credit,new car,691.0,<100,>=7,4.0,male single,none,3.0,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes


['good' 'good' 'good' 'good' 'bad']


## Model

In [8]:
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

In [9]:
model_obj = SimpleModel(seed, train_data, train_labels, test_data, test_labels, attribute_types, learner, param_grid)

In [10]:
categorical_columns = model_obj.categorical_columns
categorical_columns

['checking_status',
 'credit_history',
 'purpose',
 'savings_status',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'job',
 'own_telephone',
 'foreign_worker']

In [11]:
numerical_columms = model_obj.numerical_columms
numerical_columms

['duration',
 'credit_amount',
 'installment_commitment',
 'residence_since',
 'age',
 'existing_credits',
 'num_dependents']

In [12]:
model = model_obj.fit_baseline_model(train_data, train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   12.9s finished


## Corruptions

In [13]:
# corruption perturbations to apply
corr_perturbations = Perturbation(categorical_columns, numerical_columms)

In [14]:
test_data_corrupted, perturbations, cols_perturbed = corr_perturbations.apply_perturbation(test_data, 5)
test_data_corrupted.head(10)

GaussianNoise: {'column': 'residence_since', 'fraction': 0.25}
['residence_since']
MissingValues: {'column': 'credit_history', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MAR'}
['credit_history']
GaussianNoise: {'column': 'duration', 'fraction': 0.25}
['duration']
MissingValues: {'column': 'credit_amount', 'fraction': 0.75, 'na_value': nan, 'missingness': 'MAR'}
['credit_amount']
SwappedValues: {'column_a': 'housing', 'column_b': 'personal_status', 'fraction': 0.25}
['housing', 'personal_status']


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.000000,NaN,used car,NaN,<100,1<=X<4,3.0,male single,none,0.193852,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.000000,critical/other existing credit,radio/tv,NaN,no known savings,>=7,4.0,male single,none,4.000000,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,-3.982692,NaN,new car,1255.0,<100,>=7,4.0,male single,none,4.000000,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,-30.933795,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,2.000000,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.000000,NaN,new car,NaN,<100,>=7,4.0,male single,none,7.827427,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes
955,<0,24.000000,critical/other existing credit,radio/tv,NaN,>=1000,>=7,4.0,female div/dep/mar,none,2.852679,life insurance,57.0,none,rent,2.0,high qualif/self emp/mgmt,1.0,yes,yes
121,no checking,24.000000,critical/other existing credit,used car,NaN,<100,>=7,4.0,rent,none,2.000000,car,41.0,none,female div/dep/mar,2.0,high qualif/self emp/mgmt,1.0,yes,yes
230,>=200,36.000000,existing paid,radio/tv,4210.0,<100,1<=X<4,4.0,male single,none,5.605912,car,26.0,none,own,1.0,skilled,1.0,none,yes
11,<0,48.000000,existing paid,business,4308.0,<100,<1,3.0,female div/dep/mar,none,6.635504,life insurance,24.0,none,rent,1.0,skilled,1.0,none,yes
120,<0,21.000000,NaN,radio/tv,NaN,<100,1<=X<4,3.0,female div/dep/mar,none,2.000000,real estate,25.0,none,own,2.0,skilled,1.0,yes,yes


In [15]:
perturbations

In [44]:
cols_perturbed

['residence_since',
 'credit_history',
 'duration',
 'credit_amount',
 'housing',
 'personal_status']

### Visualize the original and corrupted test set

In [ ]:
## original test data
plot_data(pd.concat([test_data, pd.Series(test_labels, name='class')], axis=1))

In [ ]:
## corrupted test data
plot_data(pd.concat([test_data_corrupted, pd.Series(test_labels, name='class')], axis=1))

## Cleaning

### Imputation

In [17]:
mean_mode_imputer = MeanModeImputation(train_data, test_data_corrupted, categorical_columns, numerical_columms)

mean_mode_imputer.fit(train_data)
test_data_mm_imputed = mean_mode_imputer.transform(test_data_corrupted)
test_data_mm_imputed

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.000000,existing paid,used car,3297.265,<100,1<=X<4,3.0,male single,none,0.193852,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.000000,critical/other existing credit,radio/tv,3297.265,no known savings,>=7,4.0,male single,none,4.000000,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,-3.982692,existing paid,new car,1255.000,<100,>=7,4.0,male single,none,4.000000,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,-30.933795,existing paid,radio/tv,1414.000,<100,1<=X<4,4.0,male single,guarantor,2.000000,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.000000,existing paid,new car,3297.265,<100,>=7,4.0,male single,none,7.827427,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,<0,30.000000,existing paid,repairs,3297.265,<100,<1,1.0,male div/sep,none,1.000000,no known property,34.0,none,own,1.0,unskilled resident,1.0,yes,yes
192,0<=X<200,27.000000,existing paid,business,3297.265,<100,1<=X<4,4.0,male single,none,2.000000,car,36.0,none,own,1.0,skilled,2.0,yes,yes
398,0<=X<200,12.000000,existing paid,new car,3297.265,<100,>=7,1.0,rent,none,1.000000,real estate,46.0,none,male div/sep,2.0,skilled,1.0,none,yes
450,no checking,30.597022,critical/other existing credit,used car,3297.265,no known savings,1<=X<4,4.0,male single,none,5.279632,car,30.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes


In [19]:
datawig_imputer = DatawigImputation(train_data, test_data_corrupted, categorical_columns, numerical_columms)

test_data_dw_imputed = datawig_imputer.fit_transform(train_data, test_data_corrupted)
test_data_dw_imputed

2020-06-20 23:30:02,212 [INFO]  CategoricalEncoder for column checking_status                                found only 44 occurrences of value >=200


Fitting model for column: checking_status


2020-06-20 23:30:04,310 [INFO]  
========== start: fit model
2020-06-20 23:30:04,312 [WARNING]  Already bound, ignoring bind()
2020-06-20 23:30:05,088 [INFO]  Epoch[0] Batch [0-23]	Speed: 527.46 samples/sec	cross-entropy=1.270066	checking_status-accuracy=0.369792
2020-06-20 23:30:05,703 [INFO]  Epoch[0] Train-cross-entropy=1.232720
2020-06-20 23:30:05,705 [INFO]  Epoch[0] Train-checking_status-accuracy=0.373611
2020-06-20 23:30:05,708 [INFO]  Epoch[0] Time cost=1.388
2020-06-20 23:30:05,725 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-20 23:30:05,868 [INFO]  Epoch[0] Validation-cross-entropy=1.033619
2020-06-20 23:30:05,870 [INFO]  Epoch[0] Validation-checking_status-accuracy=0.550000
2020-06-20 23:30:06,579 [INFO]  Epoch[1] Batch [0-23]	Speed: 552.20 samples/sec	cross-entropy=1.128721	checking_status-accuracy=0.434896
2020-06-20 23:30:07,225 [INFO]  Epoch[1] Train-cross-entropy=1.131450
2020-06-20 23:30:07,228 [INFO]  Epoch[1] Train-checking_status-accuracy=0.

Fitting model for column: credit_history


2020-06-20 23:30:27,254 [INFO]  
========== start: fit model
2020-06-20 23:30:27,259 [WARNING]  Already bound, ignoring bind()
2020-06-20 23:30:28,047 [INFO]  Epoch[0] Batch [0-23]	Speed: 513.86 samples/sec	cross-entropy=1.085513	credit_history-accuracy=0.580729
2020-06-20 23:30:28,806 [INFO]  Epoch[0] Train-cross-entropy=1.050243
2020-06-20 23:30:28,810 [INFO]  Epoch[0] Train-credit_history-accuracy=0.627778
2020-06-20 23:30:28,814 [INFO]  Epoch[0] Time cost=1.547
2020-06-20 23:30:28,844 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-20 23:30:28,948 [INFO]  Epoch[0] Validation-cross-entropy=0.864091
2020-06-20 23:30:28,951 [INFO]  Epoch[0] Validation-credit_history-accuracy=0.700000
2020-06-20 23:30:29,749 [INFO]  Epoch[1] Batch [0-23]	Speed: 484.53 samples/sec	cross-entropy=0.867723	credit_history-accuracy=0.705729
2020-06-20 23:30:30,431 [INFO]  Epoch[1] Train-cross-entropy=0.897992
2020-06-20 23:30:30,434 [INFO]  Epoch[1] Train-credit_history-accuracy=0.70138

...

Fitting model for column: existing_credits


2020-06-20 23:37:39,841 [INFO]  
========== start: fit model
2020-06-20 23:37:39,849 [WARNING]  Already bound, ignoring bind()
2020-06-20 23:37:40,562 [INFO]  Epoch[0] Batch [0-23]	Speed: 584.94 samples/sec	cross-entropy=15.585867	existing_credits-accuracy=0.000000
2020-06-20 23:37:41,131 [INFO]  Epoch[0] Train-cross-entropy=15.560104
2020-06-20 23:37:41,137 [INFO]  Epoch[0] Train-existing_credits-accuracy=0.000000
2020-06-20 23:37:41,145 [INFO]  Epoch[0] Time cost=1.280
2020-06-20 23:37:41,163 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-20 23:37:41,278 [INFO]  Epoch[0] Validation-cross-entropy=12.469859
2020-06-20 23:37:41,283 [INFO]  Epoch[0] Validation-existing_credits-accuracy=0.000000
2020-06-20 23:37:42,061 [INFO]  Epoch[1] Batch [0-23]	Speed: 526.81 samples/sec	cross-entropy=13.436795	existing_credits-accuracy=0.000000
2020-06-20 23:37:42,619 [INFO]  Epoch[1] Train-cross-entropy=13.678627
2020-06-20 23:37:42,628 [INFO]  Epoch[1] Train-existing_credits-a

Fitting model for column: num_dependents


2020-06-20 23:38:09,057 [INFO]  
========== start: fit model
2020-06-20 23:38:09,064 [WARNING]  Already bound, ignoring bind()
2020-06-20 23:38:09,662 [INFO]  Epoch[0] Batch [0-23]	Speed: 660.34 samples/sec	cross-entropy=14.704023	num_dependents-accuracy=0.000000
2020-06-20 23:38:10,253 [INFO]  Epoch[0] Train-cross-entropy=15.495457
2020-06-20 23:38:10,260 [INFO]  Epoch[0] Train-num_dependents-accuracy=0.000000
2020-06-20 23:38:10,275 [INFO]  Epoch[0] Time cost=1.200
2020-06-20 23:38:10,302 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-20 23:38:10,479 [INFO]  Epoch[0] Validation-cross-entropy=14.296767
2020-06-20 23:38:10,493 [INFO]  Epoch[0] Validation-num_dependents-accuracy=0.000000
2020-06-20 23:38:11,386 [INFO]  Epoch[1] Batch [0-23]	Speed: 436.92 samples/sec	cross-entropy=13.424981	num_dependents-accuracy=0.000000
2020-06-20 23:38:11,912 [INFO]  Epoch[1] Train-cross-entropy=14.404770
2020-06-20 23:38:11,917 [INFO]  Epoch[1] Train-num_dependents-accuracy=0.

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.000000,nan,used car,1598.119777,<100,1<=X<4,3.0,male single,none,0.193852,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.000000,critical/other existing credit,radio/tv,2533.905632,no known savings,>=7,4.0,male single,none,4.000000,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,-3.982692,nan,new car,1255.000000,<100,>=7,4.0,male single,none,4.000000,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,-30.933795,existing paid,radio/tv,1414.000000,<100,1<=X<4,4.0,male single,guarantor,2.000000,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.000000,nan,new car,927.096854,<100,>=7,4.0,male single,none,7.827427,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,<0,30.000000,existing paid,repairs,4776.586919,<100,<1,1.0,male div/sep,none,1.000000,no known property,34.0,none,own,1.0,unskilled resident,1.0,yes,yes
192,0<=X<200,27.000000,existing paid,business,2737.398185,<100,1<=X<4,4.0,male single,none,2.000000,car,36.0,none,own,1.0,skilled,2.0,yes,yes
398,0<=X<200,12.000000,nan,new car,614.418651,<100,>=7,1.0,rent,none,1.000000,real estate,46.0,none,male div/sep,2.0,skilled,1.0,none,yes
450,no checking,30.597022,critical/other existing credit,used car,5179.101487,no known savings,1<=X<4,4.0,male single,none,5.279632,car,30.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes


In [ ]:
# for all imputers return scores, take best

### Outlier Detection

In [22]:
# detection using KNN from PyOD
outlier = PyODKNN(train_data, test_data_corrupted, categorical_columns, numerical_columms)

In [23]:
test_data_corrupted_outliers = outlier.fit_transform(train_data, test_data_corrupted)
test_data_corrupted_outliers.head(10)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,outlier
841,no checking,21.000000,nan,used car,NaN,<100,1<=X<4,3.0,male single,none,...,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes,0
956,>=200,30.000000,critical/other existing credit,radio/tv,NaN,no known savings,>=7,4.0,male single,none,...,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes,0
544,no checking,-3.982692,nan,new car,1255.0,<100,>=7,4.0,male single,none,...,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes,0
173,0<=X<200,-30.933795,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,...,real estate,33.0,none,own,1.0,skilled,1.0,none,no,1
759,<0,12.000000,nan,new car,NaN,<100,>=7,4.0,male single,none,...,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes,1
955,<0,24.000000,critical/other existing credit,radio/tv,NaN,>=1000,>=7,4.0,female div/dep/mar,none,...,life insurance,57.0,none,rent,2.0,high qualif/self emp/mgmt,1.0,yes,yes,0
121,no checking,24.000000,critical/other existing credit,used car,NaN,<100,>=7,4.0,rent,none,...,car,41.0,none,female div/dep/mar,2.0,high qualif/self emp/mgmt,1.0,yes,yes,0
230,>=200,36.000000,existing paid,radio/tv,4210.0,<100,1<=X<4,4.0,male single,none,...,car,26.0,none,own,1.0,skilled,1.0,none,yes,0
11,<0,48.000000,existing paid,business,4308.0,<100,<1,3.0,female div/dep/mar,none,...,life insurance,24.0,none,rent,1.0,skilled,1.0,none,yes,1
120,<0,21.000000,nan,radio/tv,NaN,<100,1<=X<4,3.0,female div/dep/mar,none,...,real estate,25.0,none,own,2.0,skilled,1.0,yes,yes,0


In [24]:
# detection using Isolation Forest from PyOD
outlier_if = PyODIsolationForest(train_data, test_data_corrupted, categorical_columns, numerical_columms)

In [42]:
test_data_corrupted_outliers_if = outlier_if.fit_transform(train_data, test_data_corrupted)
test_data_corrupted_outliers_if.head(10)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,outlier
841,no checking,21.000000,nan,used car,NaN,<100,1<=X<4,3.0,male single,none,...,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes,0
956,>=200,30.000000,critical/other existing credit,radio/tv,NaN,no known savings,>=7,4.0,male single,none,...,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes,1
544,no checking,-3.982692,nan,new car,1255.0,<100,>=7,4.0,male single,none,...,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes,0
173,0<=X<200,-30.933795,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,...,real estate,33.0,none,own,1.0,skilled,1.0,none,no,1
759,<0,12.000000,nan,new car,NaN,<100,>=7,4.0,male single,none,...,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes,0
955,<0,24.000000,critical/other existing credit,radio/tv,NaN,>=1000,>=7,4.0,female div/dep/mar,none,...,life insurance,57.0,none,rent,2.0,high qualif/self emp/mgmt,1.0,yes,yes,1
121,no checking,24.000000,critical/other existing credit,used car,NaN,<100,>=7,4.0,rent,none,...,car,41.0,none,female div/dep/mar,2.0,high qualif/self emp/mgmt,1.0,yes,yes,0
230,>=200,36.000000,existing paid,radio/tv,4210.0,<100,1<=X<4,4.0,male single,none,...,car,26.0,none,own,1.0,skilled,1.0,none,yes,0
11,<0,48.000000,existing paid,business,4308.0,<100,<1,3.0,female div/dep/mar,none,...,life insurance,24.0,none,rent,1.0,skilled,1.0,none,yes,0
120,<0,21.000000,nan,radio/tv,NaN,<100,1<=X<4,3.0,female div/dep/mar,none,...,real estate,25.0,none,own,2.0,skilled,1.0,yes,yes,0


#### Preparing the outliers for imputation

In [28]:
if 'outlier' in test_data_corrupted_outliers_if.columns:
    print(test_data_corrupted_outliers_if['outlier'].sum())

58


In [ ]:
## train_data, test_data_corrupted, 
## check values in column in the training data -> check for outliers in the same column in the corrupted data
## store .loc 
## convert those .loc for those column into nan
## impute

In [50]:
numerical_columms

['duration',
 'credit_amount',
 'installment_commitment',
 'residence_since',
 'age',
 'existing_credits',
 'num_dependents']

In [49]:
test_data_corrupted

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.000000,nan,used car,NaN,<100,1<=X<4,3.0,male single,none,0.193852,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.000000,critical/other existing credit,radio/tv,NaN,no known savings,>=7,4.0,male single,none,4.000000,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,-3.982692,nan,new car,1255.0,<100,>=7,4.0,male single,none,4.000000,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,-30.933795,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,2.000000,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.000000,nan,new car,NaN,<100,>=7,4.0,male single,none,7.827427,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,<0,30.000000,existing paid,repairs,NaN,<100,<1,1.0,male div/sep,none,1.000000,no known property,34.0,none,own,1.0,unskilled resident,1.0,yes,yes
192,0<=X<200,27.000000,existing paid,business,NaN,<100,1<=X<4,4.0,male single,none,2.000000,car,36.0,none,own,1.0,skilled,2.0,yes,yes
398,0<=X<200,12.000000,nan,new car,NaN,<100,>=7,1.0,rent,none,1.000000,real estate,46.0,none,male div/sep,2.0,skilled,1.0,none,yes
450,no checking,30.597022,critical/other existing credit,used car,NaN,no known savings,1<=X<4,4.0,male single,none,5.279632,car,30.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes


## Evaluation

In [45]:
# score without cleaning
model_obj.score_on_test_data(model.predict_proba(test_data))

0.8093735390369332

In [46]:
# score with corruptions
model_obj.score_on_test_data(model.predict_proba(test_data_corrupted))

0.8020102851799906

In [47]:
# score with mean/mode imputation
model_obj.score_on_test_data(model.predict_proba(test_data_mm_imputed))

0.791374474053296

In [48]:
# score with datawig imputation
model_obj.score_on_test_data(model.predict_proba(test_data_dw_imputed))

0.7910238429172511